<a href="https://colab.research.google.com/github/Yash-Kamtekar/Special-Topics-Assignment-5/blob/main/active_learning_with_lightly_ai_end2end.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/alexeygrigorev/clothing-dataset-small.git

Cloning into 'clothing-dataset-small'...
remote: Enumerating objects: 3839, done.
remote: Counting objects: 100% (400/400), done.
remote: Compressing objects: 100% (400/400), done.
remote: Total 3839 (delta 9), reused 385 (delta 0), pack-reused 3439
Receiving objects: 100% (3839/3839), 100.58 MiB | 32.35 MiB/s, done.
Resolving deltas: 100% (10/10), done.
Checking out files: 100% (3783/3783), done.


In [2]:
# Install lightly as a pip package
!pip install lightly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 507 kB 10.5 MB/s 
     |████████████████████████████████| 798 kB 58.3 MB/s 
     |████████████████████████████████| 151 kB 61.1 MB/s 
     |████████████████████████████████| 79 kB 8.6 MB/s 
     |████████████████████████████████| 117 kB 64.7 MB/s 
     |████████████████████████████████| 529 kB 29.6 MB/s 
     |████████████████████████████████| 87 kB 3.5 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144576 sha256=708acc6593e8f838e060d365e3a2b1da57443702f51b12afa81aef6f0ac3e953
  Stored in directory: /root/.cache/pip/wheels/8b/8d/53/2af8772d9aec614e3fc65e53d4a993ad73c61daa8bbd85a873
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115940 sha256=af3ab38d2a8417abfb5457bfa3c09f9539cd01479ed2c310d006bdb970293c8f
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e

In [3]:
# Your personal token for accessing the Lightly Platform is defined. You can get it from
# the Lightly platform at https://app.lightly.ai under your username and then Preferences.
LIGHTLY_TOKEN="7d9e3e942dbb8949325f955a838e79d822c3405cff6e0444"

In [4]:
!lightly-train input_dir=clothing-dataset-small/train trainer.max_epochs=1

/usr/local/lib/python3.7/dist-packages/hydra/_internal/hydra.py:127: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/next/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  configure_logging=with_log_configuration,
/usr/local/lib/python3.7/dist-packages/lightly/cli/train_cli.py:69: UserWarning: Training a self-supervised model with a small batch size: 16! Small batch size may harm embedding quality. You can specify the batch size via the loader key-word: loader.batch_size=BSZ
  warnings.warn(msg)
Downloading: "https://storage.googleapis.com/models_boris/whattolabel-resnet18-simclr-d32-w1.0-i-085d0693.pth" to /root/.cache/torch/hub/checkpoints/whattolabel-resnet18-simclr-d32-w1.0-i-085d0693.pth
100% 42.8M/42.8M [00:01<00:00, 27.8MB/s]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/lightning.py:23: LightningDeprecationWarning: pytorch_lightning.core.lightning.LightningModul

In [5]:
!lightly-embed input_dir=clothing-dataset-small/train checkpoint=/content/lightly_outputs/2022-11-16/06-10-07/lightly_epoch_0.ckpt

/usr/local/lib/python3.7/dist-packages/hydra/_internal/hydra.py:127: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/next/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  configure_logging=with_log_configuration,
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/lightning.py:23: LightningDeprecationWarning: pytorch_lightning.core.lightning.LightningModule has been deprecated in v1.7 and will be removed in v1.9. Use the equivalent class from the pytorch_lightning.core.module.LightningModule class instead.
  "pytorch_lightning.core.lightning.LightningModule has been deprecated in v1.7"
Compute efficiency: 0.16: 100% 3068/3068 [00:13<00:00, 232.62imgs/s]
Embeddings are stored at /content/lightly_outputs/2022-11-16/06-11-14/embeddings.csv


In [6]:
# rename the embeddings output to embeddings.csv
!mv /content/lightly_outputs/2022-11-16/06-11-14/embeddings.csv embeddings.csv

In [7]:
# Install the pip packages required for the tutorial if not already installed
!pip install numpy
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
# In the console output of the lightly-magic command, you find the filename of the created
# embeddings file. We need this file later, so set the path to it as an environment variable.
LIGHTLY_EMBEDDINGS_CSV="embeddings.csv"

In [9]:
import os
import csv
from typing import List, Dict, Tuple
import numpy as np
from sklearn.linear_model import LogisticRegression

from lightly.active_learning.agents.agent import ActiveLearningAgent
from lightly.active_learning.config.selection_config import SelectionConfig
from lightly.active_learning.scorers.classification import ScorerClassification
from lightly.api.api_workflow_client import ApiWorkflowClient
from lightly.openapi_generated.swagger_client import SamplingMethod

In [10]:
token = "7d9e3e942dbb8949325f955a838e79d822c3405cff6e0444"
path_to_embeddings_csv = LIGHTLY_EMBEDDINGS_CSV
# We define the client to the Lightly Platform API
api_workflow_client = ApiWorkflowClient(token=token)
api_workflow_client.create_dataset(dataset_name="active_learning_clothing_dataset")

In [11]:
path_to_embeddings_csv = "embeddings.csv"
YOUR_TOKEN = "7d9e3e942dbb8949325f955a838e79d822c3405cff6e0444"  # your token of the web platform
YOUR_DATASET_ID = "63747f4896ba509f0cd0aa62"  # the id of your dataset on the web platform

In [12]:
#run later
!lightly-magic token='7d9e3e942dbb8949325f955a838e79d822c3405cff6e0444' dataset_id='63747f4896ba509f0cd0aa62' input_dir='clothing-dataset-small/train' trainer.max_epochs=20

#!LIGHTLY_SERVER_LOCATION='https://api-dev.lightly.ai' lightly-upload input_dir="clothing-dataset-small/train" token="8fe1f28d710558277de10fa50e31dca637f8e97306cdf3cf" dataset_id="636ef6d6e2cafea9f47cc585"

/usr/local/lib/python3.7/dist-packages/hydra/_internal/hydra.py:127: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/next/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  configure_logging=with_log_configuration,
########## Starting to train an embedding model.
/usr/local/lib/python3.7/dist-packages/lightly/cli/train_cli.py:69: UserWarning: Training a self-supervised model with a small batch size: 16! Small batch size may harm embedding quality. You can specify the batch size via the loader key-word: loader.batch_size=BSZ
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/lightning.py:23: LightningDeprecationWarning: pytorch_lightning.core.lightning.LightningModule has been deprecated in v1.7 and will be removed in v1.9. Use the equivalent class from the pytorch_lightning.core.module.LightningModule class instead.
  "pytorch_lightning.core.lightning.Lig

In [13]:
class CSVEmbeddingDataset:
    def __init__(self, path_to_embeddings_csv: str):
        with open(path_to_embeddings_csv, 'r') as f:
            data = csv.reader(f)

            rows = list(data)
            header_row = rows[0]
            rows_without_header = rows[1:]

            index_filenames = header_row.index('filenames')
            filenames = [row[index_filenames] for row in rows_without_header]

            index_labels = header_row.index('labels')
            labels = [row[index_labels] for row in rows_without_header]

            embeddings = rows_without_header
            indexes_to_delete = sorted([index_filenames, index_labels], reverse=True)
            for embedding_row in embeddings:
                for index_to_delete in indexes_to_delete:
                    del embedding_row[index_to_delete]

        # create the dataset as a dictionary mapping from the filename to a tuple of the embedding and the label
        self.dataset: Dict[str, Tuple[np.ndarray, int]] = \
            dict([(filename, (np.array(embedding_row, dtype=float), int(label)))
                  for filename, embedding_row, label in zip(filenames, embeddings, labels)])

    def get_features(self, filenames: List[str]) -> np.ndarray:
        features_array = np.array([self.dataset[filename][0] for filename in filenames])
        return features_array

    def get_labels(self, filenames: List[str]) -> np.ndarray:
        labels = np.array([self.dataset[filename][1] for filename in filenames])
        return labels

In [14]:
dataset = CSVEmbeddingDataset(path_to_embeddings_csv=path_to_embeddings_csv)
classifier = LogisticRegression(max_iter=1000)
agent = ActiveLearningAgent(api_workflow_client=api_workflow_client)

In [15]:
print("Starting the initial selection")
selection_config = SelectionConfig(n_samples=200, method=SamplingMethod.CORESET, name='initial-selection')
agent.query(selection_config=selection_config)
print(f"There are {len(agent.labeled_set)} samples in the labeled set.")

Starting the initial selection
There are 200 samples in the labeled set.


In [16]:
labeled_set_features = dataset.get_features(agent.labeled_set)
labeled_set_labels = dataset.get_labels(agent.labeled_set)
classifier.fit(X=labeled_set_features, y=labeled_set_labels)

LogisticRegression(max_iter=1000)

In [17]:
query_set_features = dataset.get_features(agent.query_set)
predictions = classifier.predict_proba(X=query_set_features)

In [18]:
active_learning_scorer = ScorerClassification(model_output=predictions)

In [19]:
selection_config = SelectionConfig(n_samples=300, method=SamplingMethod.CORAL, name='al-iteration-1')
agent.query(selection_config=selection_config, al_scorer=active_learning_scorer)
print(f"There are {len(agent.labeled_set)} samples in the labeled set.")

There are 300 samples in the labeled set.


In [20]:
labeled_set_features = dataset.get_features(agent.labeled_set)
labeled_set_labels = dataset.get_labels(agent.labeled_set)
classifier.fit(X=labeled_set_features, y=labeled_set_labels)

# evaluate on unlabeled set
unlabeled_set_features = dataset.get_features(agent.unlabeled_set)
unlabeled_set_labels = dataset.get_labels(agent.unlabeled_set)
accuracy_on_unlabeled_set = classifier.score(X=unlabeled_set_features, y=unlabeled_set_labels)
print(f"accuracy on unlabeled set: {accuracy_on_unlabeled_set}")

accuracy on unlabeled set: 0.3800578034682081
